# Replaced by connector_reliefweb_disaster_summaries on 2/6/2024

# Parse the json files that have been saved to disk



In [86]:
import requests
import pandas as pd
import uuid
import json
import os
import uuid
import re

from datetime import datetime

In [87]:
def parse_summary_json(j):
    reference_url = ''
    file_url = ''
    themes = []
    author_org = ''
    
    rec_id = j['id']
    j = j['fields']

    status = j['status']
    glide_id = j.get('glide')
    status = j['status']
    title = j['name']
    description = j['description']
    file_url = j['url_alias']
    primary_country_iso3 = j['primary_country']['iso3']
    primary_country = j['primary_country']['shortname']
    report_date = j['date']['changed']
   
    original_text_list = description.split("\n\n")
    idx_para=0
    for o in original_text_list:
        o = o.strip()
        row = ['disaster summary',status,file_url,glide_id,idx_para,primary_country,title,themes,o,reference_url,o,author_org,report_date]
        df_rw_disaster_sum.loc[len(df_rw_disaster_sum)] = row
        idx_para += 1


def parse_references(text):
    ref = ''
    org = ''
    date = ''
    iso_date = None
    url = ''
    
    pattern = r'\([^)]+\)\)\s?'  # Matches anything inside a ([xxxx](url)) pattern

    ref = re.findall(pattern, text)
    if len(ref) > 0:
        last_reference = ref[-1]
        #pattern = r'\[(.*?)[,|.]\s(\d+\s\w+\s\d+)\]\((.*?)\)'
        pattern = r'\[(.*?)[,|.]\s?(\d+\s\w+\s\d+)\]\((.*?)\)'

        # Use regex to find matches
        matches = re.search(pattern, last_reference)
        
        if matches:
            # Extracting the parts
            org = matches.group(1)
            date = matches.group(2)
            url = matches.group(3)
        
            
            try:
                date_object = datetime.strptime(date, "%d %b %Y") # 01 Jul 2023
                iso_date = date_object.date().isoformat()
            except:
                try:
                    date_object = datetime.strptime(date, "%d %B %Y") # 01 July 2023
                    iso_date = date_object.date().isoformat()
                except:
                    pass

            #date_object = datetime.strptime(date, "%d %b %Y")
           
            
        else:
            print(f"No match found. {ref}")

    
    return pd.Series({'references':ref,'auth_org':org,'date_str':date,'date_iso':iso_date,'reference_url':url})






In [88]:
#prep receiving df
df_rw_disaster_sum = pd.DataFrame(columns = ['record_type','status','source_url','glide_id','idx_para','source_level_country','source_title','source_desc',
                                                        'source_original_text','reference_url','text','authoring_org','reported_date'])

# get files on disk
fpath = "D://projects//_external_files//reliefweb_disaster_reports//"
files_on_disk = {}
for f in os.listdir(fpath):
    k = f.split('_')[0]
    files_on_disk[k]=f

#file = files_on_disk['51520']
for f in files_on_disk:
    with open(f"{fpath}{files_on_disk[f]}", 'r') as file:
        json_data = json.load(file)
        parse_summary_json(json_data)

df_rw_disaster_sum['authoring_org'] = 'reliefweb'
df_rw_disaster_sum[['references','reference_auth_org','reference_date_str','reference_date_iso','reference_url']] = df_rw_disaster_sum['text'].apply(parse_references) 
df_rw_disaster_sum['para_id'] = 'rwdisastersumm_' + df_rw_disaster_sum['glide_id'].apply(lambda x: x.lower()) + '_' + df_rw_disaster_sum['idx_para'].astype(str)

def generate_uuid(x):
    return uuid.uuid4().hex
    
output_file = f"D://projects//_external_files//surveyor//rw_disaster_preprocessed//disaster_summaries_json_parsed_{generate_uuid(1)}.xlsx"
df_rw_disaster_sum.to_excel(output_file, sheet_name="json_parse", index=False)

No match found. ['([UNICEF, 31 Dec 2019.](https://reliefweb.int/node/3493448))']
No match found. ['([IOM, 16 Dec 2016.](https://reliefweb.int/node/1824524))']
No match found. ['([UNICEF, 16 Dec 2016.](https://reliefweb.int/node/1824734))']
No match found. ["([Gov't Nigeria, 2 Dec 2018.](https://reliefweb.int/node/2906648)) ", "([Gov't Nigeria, 2 Dec 2018.](https://reliefweb.int/node/2906663))"]
No match found. ['([WHO, 29 Sep 2019.](https://reliefweb.int/node/3330078))']
No match found. ['([WHO, 05 Jan 2020.](https://reliefweb.int/node/3464608))']
No match found. ['([Govt Nigeria, 30 Mar 2020.](https://reliefweb.int/node/3727212))']
No match found. ['([GPEI 13 May 2020](https://reliefweb.int/report/pakistan/polio-week-13-may-2020))']
No match found. ['(https://reliefweb.int/node/3980536))']
No match found. ['([IFRC 2 Feb 2020](https://reliefweb.int/node/3507329))']
No match found. ['(ECHO, 2 Apr 2020](https://reliefweb.int/node/3588843))']
No match found. ['(ECHO, 3 Apr 2020](https://r

TypeError: NDFrame.to_excel() got an unexpected keyword argument 'encoding'

In [ ]:
print(output_file)